# A brief summary of *OSEK COM*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Data Types](#data-types)
* [System Calls](#system-calls)
  * [Start-up Services](#start-up-services)
    * [`StartCOM`](#startcom)
    * [`StopCOM`](#stopcom)
    * [`GetCOMApplicationMode`](#getcomapplicationmode)
    * [`InitMessage`](#initmessage)
    * [`StartPeriodic`](#startperiodic)
    * [`StopPeriodic`](#stopperiodic)
  * [Notification Services](#notification-services)
    * [`ReadFlag`](#readflag)
    * [`ResetFlag`](#resetflag)
  * [Communication Services](#communication-services)
    * [`SendMessage`](#sendmessage)
    * [`ReceiveMessage`](#receivemessage)
    * [`SendDynamicMessage`](#senddynamicmessage)
    * [`ReceiveDynamicMessage`](#receivedynamicmessage)
    * [`SendZeroMessage`](#sendzeromessage)
    * [`GetMessageStatus`](#getmessagestatus)
    * [`COMErrorGetServiceId`](#comerrorgetserviceid)
  * [User-provided Routines](#user-provided-routines)
    * [`StartCOMExtension`](#startcomextension)
    * [`COMCallout(ROUTINE-NAME)`](#comcallout-routine-name)
    * [`COMErrorHook`](#comerrorhook)
* [Specification](#specification)
  * [Transmission](#transmission)
    * [Internal Communication](#internal-communication)
    * [External Communication](#external-communication)
      * [*Minimum Delay Time (MDT)*](#minimum-delay-time-mdt)
  * [Reception](#reception)
  * [Message Filtering](#message-filtering)
    * [Reception Filtering](#reception-filtering)
    * [Transmission Filtering](#transmission-filtering)
  * [Notification](#notification)
    * [Notification Mechanisms](#notification-mechanisms)
  * [Deadline Monitoring](#deadline-monitoring)
    * [Reception Deadline Monitoring](#reception-deadline-monitoring)
    * [Transmission Deadline Monitoring](#transmission-deadline-monitoring)
  * [Callouts](#callouts)
  * [Dynamic and Zero-length Messages](#dynamic-and-zero-length-messages)
    * [Dynamic Messages](#dynamic-messages)
    * [Zero-length Messages](#zero-length-messages)


Before reading this document, sift through a *Brief Summary of OSEK OS*.

## Data Types <a class="anchor" id="data-types"></a>

| | |
| --- | --- |
| *Name* | `MessageIdentifier` |
| *Description* | This type represents the identifier of a message object.

| | |
| --- | --- |
| *Name* | `ApplicationDataRef` |
| *Description* | This pointer type references data in the application.

| | |
| --- | --- |
| *Name* | `COMLengthType` |
| *Description* | This type represents length, of data, in general.

| | |
| --- | --- |
| *Name* | `LengthRef` |
| *Description* | This pointer type is a `COMLengthType` reference.

| | |
| --- | --- |
| *Name* | `FlagValue` |
| *Description* | This type represents a boolean flag, with either `COM_TRUE` or `COM_FALSE` set.

| | |
| --- | --- |
| *Name* | `COMApplicationModeType` |
| *Description* | This type represents a COM-specific application mode.

| | |
| --- | --- |
| *Name* | `COMShutdownModeType` |
| *Description* | This type represents a COM-specific shutdown mode.

| | |
| --- | --- |
| *Name* | `CalloutReturnType` |
| *Description* | This type usually represents a boolean flag, with either `COM_TRUE` or `COM_FALSE` set. <br> It is returned by COM callout functions.

| | |
| --- | --- |
| *Name* | `COMServiceIdType` |
| *Description* | This type represents the identifier of a COM service (i.e, system call).

*Note:* All COM service identifiers are available as macros of the following format: `COMServiceId_UNIQUE-NAME`.

## System Calls <a class="anchor" id="system-calls"></a>

### Start-up Services <a class="anchor" id="start-up-services"></a>

#### `StartCOM` <a class="anchor" id="startcom"></a>

| | |
| --- | --- |
| *API* | `StartCOM(COMApplicationModeType mode)` |
| *Description* | This API initializes the *OSEK COM* implementation. <br> Implementation-specific (i.e, hardware-related) settings must be configured before calling `StartCOM(..)`.

*Note:* For an *OSEK-compliant* OS, `StartCOM(..)` must be called from within a task.

*Note:* Before returning, `StartCOMExtension(..)`, a user-defined routine, is called.

*Note:* `StartCOM(..)` does not enable the periodic transmission of messages (i.e, call `StartPeriodic(..)`). Optionally, `StartCOMExtension(..)` may perform this.

#### `StopCOM` <a class="anchor" id="stopcom"></a>

| | |
| --- | --- |
| *API* | `StopCOM(COMShutdownModeType mode)` |
| *Description* | This API shuts down the *OSEK COM* implementation. <br> If successfull, `StartCOM(..)` may be called to re-initialize the implementation.

*Note:* *OSEK COM* defines `COM_SHUTDOWN_IMMEDIATE` as a `ShutdownModeType`, whereby communication activity ceases immediately.

#### `GetCOMApplicationMode` <a class="anchor" id="getcomapplicationmode"></a>

| | |
| --- | --- |
| *API* | `COMApplicationModeType GetCOMApplicationMode()` |
| *Description* | This API returns the current COM application mode. <br> It must be called after `StartCOM` has been called (i.e, `StartCOMExtension(..)` may call it).

#### `InitMessage` <a class="anchor" id="initmessage"></a>

| | |
| --- | --- |
| *API* | `InitMessage(MessageIdentifier id, ApplicationDataRef dataRef)` |
| *Description* | This API is used to initialize a message object explicitly, usually called from `StartCOMExtension`. <br><br> For dynamic messages, the length is assumed to be maximum. <br> For transmitted, external messages, the message is updated in the associated *I-PDU* buffer, after calling cpu-order and network-order message callouts, if configured. <br> For received, queued messages, the queue length is set to zero. <br> For received, unqueued messages, the respective buffer is updated.

#### `StartPeriodic` <a class="anchor" id="startperiodic"></a>

| | |
| --- | --- |
| *API* | `StartPeriodic()` |
| *Description* | This API starts periodic transmission of all messages using either the *Periodic* or *Mixed* transmission modes. <br> A period and an initial offset are configured per I-PDU.

#### `StopPeriodic` <a class="anchor" id="stopperiodic"></a>

| | |
| --- | --- |
| *API* | `StopPeriodic()` |
| *Description* | This API stops periodic transmission of all messages using either the *Periodic* or *Mixed* transmission modes. <br> If successful, `StartPeriodic(..)` may be called to restart the transmission.

### Notification Services <a class="anchor" id="notification-services"></a>

#### `ReadFlag` <a class="anchor" id="readflag"></a>

| | |
| --- | --- |
| *API* | `FlagValue ReadFlag_UNIQUE-NAME()` |
| *Description* | Reads the flag value belonging to a specific message object, and notification class it demands.

#### `ResetFlag` <a class="anchor" id="resetflag"></a>

| | |
| --- | --- |
| *API* | `FlagValue ResetFlag_UNIQUE-NAME()` |
| *Description* | Resets the flag value belonging to a specific message object, and notification class it demands.

### Communication Services <a class="anchor" id="communication-services"></a>

#### `SendMessage` <a class="anchor" id="sendmessage"></a>

| | |
| --- | --- |
| *API* | `SendMessage(MessageIdentifier id, ApplicationDataRef dataRef)` |
| *Description* | This API sends message with identifier `id`, referenced in `dataRef`. For external communication, the message is updated in its respective I-PDU buffer, and transmission is initiated (or not) according to the transfer property and transmission mode of the message object and its associated I-PDU, respectively.<br><br> For internal communication, the message is received directly by the receiving message object.

*Note:* Additionally, `SendMessage(..)` resets all relevant notification flags.

#### `ReceiveMessage` <a class="anchor" id="receivemessage"></a>

| | |
| --- | --- |
| *API* | `ReceiveMessage(MessageIdentifier msg, ApplicationDataRef dataRef)` |
| *Description* | This API receives message with identifier `id` into `dataRef`. <br><br> For queued message objects, `E_COM_NOMSG` is returned if queue is empty. If `E_COM_LIMIT` is returned, an overflow occured (i.e, received message discarded due to queue being full) since the last call, but data is dequeued in `dataRef` as expected.

*Note:* Additionally, `ReceiveMessage(..)` resets all relevant notification flags.

#### `SendDynamicMessage` <a class="anchor" id="senddynamicmessage"></a>

| | |
| --- | --- |
| *API* | `SendDynamicMessage(MessageIdentifier msg, ApplicationDataRef dataRef, LengthRef lenRef)` |
| *Description* | Similar to `SendMessage(..)`, but may only refer to unqueued message objects, and for external communication only.

#### `ReceiveDynamicMessage` <a class="anchor" id="receivedynamicmessage"></a>

| | |
| --- | --- |
| *API* | `ReceiveDynamicMessage(MessageIdentifier msg, ApplicationDataRef dataRef, LengthRef lenRef)` |
| *Description* | Similar to `ReceiveMessage(..)`, but may only refer to unqueued message objects, and for external communication only.

#### `SendZeroMessage` <a class="anchor" id="sendzeromessage"></a>

| | |
| --- | --- |
| *API* | `SendZeroMessage(MessageIdentifier msg)` |
| *Description* | Similar to `SendMessage(..)`.

#### `GetMessageStatus` <a class="anchor" id="getmessagestatus"></a>

| | |
| --- | --- |
| *API* | `GetMessageStatus(MessageIdentifier msg)` |
| *Description* | This API may be used for queued message objects only, and returns the same status values as `SendMessage(..)`, without possibly dequeuing.

#### `COMErrorGetServiceId` <a class="anchor" id="comerrorgetserviceid"></a>

| | |
| --- | --- |
| *API* | `COMServiceIdType COMErrorGetServiceId()` |
| *Description* | This API returns the identifier of the COM service where an error occured, to be called from within `COMErrorHook` only.

### User-provided Routines <a class="anchor" id="user-provided-routines"></a>

#### `StartCOMExtension` <a class="anchor" id="startcomextension"></a>

| | |
| --- | --- |
| *API* | `void StartCOMExtension(StatusType status)` |
| *Description* | This function is called by `StartCOM` before returning.

#### `COMCallout(ROUTINE-NAME)` <a class="anchor" id="comcallout-routine-name"></a>

| | |
| --- | --- |
| *API* | `void COMCallout(ROUTINE-NAME)` |
| *Description* | This function is called by *OSEK COM* at a specific point in the transmission or reception sequence, depending on configuration (see below). <br> The return value, whether `COM_TRUE` or `COM_FALSE`, indicates whether the *IL* shall continue or abandon further processing of this message or *I-PDU*.

#### `COMErrorHook` <a class="anchor" id="comerrorhook"></a>

| | |
| --- | --- |
| *API* | `void COMErrorHook(StatusType status)` |
| *Description* | This function is called at the end of a system service that returns non-`E_OK` status. <br> It is not interruptible by ISR Category-2.

## Specification <a class="anchor" id="specification"></a>

Communication in *OSEK COM* operates in the *Interaction Layer (IL)*, and is based on *messages*. A message contains application-specific data.

Administration of messages is done based on *message objects*. Transmitting message objects are on the transmitting side, and vice versa.

Multiple senders, in the application, may write to a single transmitting message object, and similarly, multiple receivers may read from a single receiving message object.

A single transmitting message object may map to multiple receiving message objects, internal and external, but not the other way around.

### Transmission <a class="anchor" id="transmission"></a>

#### Internal Communication <a class="anchor" id="internal-communication"></a>

For messages intended for internal communication, when a sender writes to a transmitting message object, messages are directly routed to the mapped receiving message objects, and placed in their respective buffers.

#### External Communication <a class="anchor" id="external-communication"></a>

For messages intended for external communication, messages must be configured to map to an *Interaction Protocol Data Unit (I-PDU)*. An *I-PDU* consists of one, or more messages. Each message occupies a continguous number of bits in an *I-PDU*. A configurable initial value is given to every message in an *I-PDU*. When a sender writes to a transmitting message object, messages are placed into their *I-PDU*'s buffer. Whether the *I-PDU* is relayed to lower layers of communication for transmission, depends on:
* The *transfer property* of the respective message.
* The *transmission mode* of its respective *I-PDU*.

Messages have a transfer property, which may be *triggered* or *pending*.

| *Transfer Property* | *Description* |
| --- | --- |
| Triggered | When placed into an I-PDU, with a non-*periodic* transmission mode, it triggers the transmission of that I-PDU to the lower layers of communication.
| Pending | When placed into an I-PDU, it does not trigger the transmission of that I-PDU to the lower layers of communication.

I-PDUs have a transmission mode, which may be *direct*, *periodic*, and *mixed*.

| *Transmission Mode* | *Description* |
| --- | --- |
| Direct | Transmission of an I-PDU is initiated, when a mapped message with *triggered* transfer property is sent.
| Periodic | Transmission of an I-PDU is initiated, with a fixed period between each transmission.
| Mixed | Transmission of an I-PDU is initiated, periodically, as well as, directly.

##### *Minimum Delay Time (MDT)* <a class="anchor" id="minimum-delay-time-mdt"></a>

A *Minimum Delay Time (MDT)* is configured per *I-PDU*, representing the minimum waiting time before the next *I-PDU* may be transmitted.

In direct or mixed transmission mode, writing multiple times to a transmitting message object within an *MDT* triggers only a single transmission of the associated *I-PDU*, after the *MDT* ends.

It is also possible for *MDT* to delay the periodic transmission of a mixed *I-PDU*.

### Reception <a class="anchor" id="reception"></a>

A receiving message object may be *queued*, or *unqueued*.

For *unqueued* message objects, the buffer is overwritten with the arrival of new messages. An initial value is set for the buffer.

For *queued* messages, each receiver of a message, in the application, reads from a dedicated queue buffer. Hence, a new message is queued into, possibly, multiple queue buffers.

### Message Filtering <a class="anchor" id="message-filtering"></a>

| *Filter Name* | *Algorithm* |
| --- | --- |
| `F_Always` | `true`
| `F_Never` | `false`
| ㅤ | 
| `F_NewIsEqual` | `new == old`
| `F_NewIsDifferent` | `new != old`
| ㅤ | 
| `F_MaskedNewEqualsMaskedOld` | `(new & mask) == (old & mask)`
| `F_MaskedNewDiffersMaskedOld` | `(new & mask) != (old & mask)`
| ㅤ | 
| `F_MaskedNewEqualsX` | `(new & mask) == x`
| `F_MaskedNewDiffersX` | `(new & mask) != x`
| ㅤ | 
| `F_NewIsWithin` | `new >= min && new <= max`
| `F_NewIsOutside` | `new < min \|\| new > max`
| `F_NewIsGreater` | `new > old`
| `F_NewIsGreaterOrEqual` | `new >= old`
| `F_NewIsLess` | `new < old`
| `F_NewIsLessOrEqual` | `new <= old`
| ㅤ | 
| `F_OneEveryN` | `occurence % period == offset`

#### Reception Filtering <a class="anchor" id="reception-filtering"></a>

A *filter algorithm* may be configured, per receiving message object, to filter out new messages upon arrival.

#### Transmission Filtering <a class="anchor" id="transmission-filtering"></a>

A *filter algorithm* may be configured, per transmitting message object, to filter out *external* messages, before the associated *I-PDU* buffer is updated, and transmission, possibly, takes place.

### Notification <a class="anchor" id="notification"></a>

*OSEK COM* defines four notification classes.

| *Class* | *Name* | *Description* |
| --- | --- | --- |
| 1 | Message Reception | Notification mechanism invoked immediately after the message has been stored in the receiving message object.
| 2 | Message Transmission | Notification mechanism invoked immediately after the successful transmission of the associated I-PDU by the lower layers of communication, conveyed via a confirmation callback function to *OSEK COM*.
| 3 | Message Reception Error | Notification mechanism invoked immediately after a message reception error has been detected, either by reception deadline monitoring, or via an error detected by the lower layers of communication, conveyed via an indication callback function to *OSEK COM*.
| 4 | Message Transmission Error | Notification mechanism invoked immediately after a message transmission error has been detected,  either by transmission deadline monitoring, or via an error detected by the lower layers of communication, conveyed via a confirmation callback function to *OSEK COM*.

*Note:* For internal communication, only *notification class 1* is supported.

*Note:* If a message is filtered out (i.e, discarded), no notification is given to the application.

#### Notification Mechanisms <a class="anchor" id="notification-mechanisms"></a>

*OSEK COM* provides four different notification mechanisms:
* Calling a *callback* routine.
* Setting a *flag*, that can be polled using `ReadFlag-`.
* Activating a task.
* Setting an event.

### Deadline Monitoring <a class="anchor" id="deadline-monitoring"></a>

#### Reception Deadline Monitoring <a class="anchor" id="reception-deadline-monitoring"></a>

Reception deadline monitoring ensures that an external message is received within a time frame, conveyed from the lower layers of communication via an *indication* service. It is configurable per *I-PDU*, and is not limited to *I-PDUs* with *periodic* transmission modes only.

#### Transmission Deadline Monitoring <a class="anchor" id="transmission-deadline-monitoring"></a>

Transmission deadline monitoring ensures that an external message is sent within a time frame, converyed from the lower layers of communication via a *confirmation* serivce. It is configurable per *I-PDU*.

### Callouts <a class="anchor" id="callouts"></a>

A *callout* is a user-provided routine, that is called by *OSEK COM* at a specific point in the transmission or reception sequence, depending on configuration (see below). 

The return value, whether `COM_TRUE` or `COM_FALSE`, indicates whether the *IL* shall continue or abandon further processing of this message or *I-PDU*.

The figure below shows the different callout functions that are called, during message reception.

<img src="../../.img/osek-com-reception.png" width="750" />

The figure below shows the different callout functions that are called, during message transmission.

<img src="../../.img/osek-com-transmission.png" width="750" />

### Dynamic and Zero-length Messages <a class="anchor" id="dynamic-and-zero-length-messages"></a>

#### Dynamic Messages <a class="anchor" id="dynamic-messages"></a>

*OSEK COM* supports *dynamic* (i.e, *dynamic*) messages, with a maximum length configured.

*Note:* Dynamic messages must be mapped to the end of an *I-PDU*, in case of external communication.

*Note:* Filter algorithms may not be applied to dynamic messages.

#### Zero-length Messages <a class="anchor" id="zero-length-messages"></a>

*OSEK COM* supports zero-length messages, useful as a notification mechanism between senders and receivers in the application.